In [56]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from requests.auth import HTTPBasicAuth
import random 
import time


In [57]:
consumer_key = 'Lq3V1hAxII2Fv8YSdcrhfA538HzhsfzhelYqSUgzGU0A8AEU'
consumer_secret_key = 'jHJjRSzVQBjCizDYmFg4cqWza2R8TOCcLaecRbXrJQcseLu8Lr0AqYXL1ldkAx8A'

# EPO OAuth2 token URL
token_url = "https://ops.epo.org/3.2/auth/accesstoken"

def get_token():
    # Request access token
    response = requests.post(
        token_url,
        auth=HTTPBasicAuth(consumer_key, consumer_secret_key),
        data={"grant_type": "client_credentials"},
    )
    
    # Extract token
    if response.status_code == 200:
        token = response.json()["access_token"]
        return token
    else:
        print("Failed to retrieve access token.")
        return None

In [58]:
data_to_collect = pd.read_csv('sampled_patents.csv')

columns = ['year', 'patent', 'name_of_patenter', 'country', 'doc_number', 'categorization']
collected_df = pd.DataFrame(columns=columns)


In [59]:
def get_patent_info(patent_number, token):
    # Base URL for constituent search
    search_url = "https://ops.epo.org/3.2/rest-services/published-data/search/biblio"

    # Headers with authentication
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/xml",
        "User-Agent": "ProjectAcademicBot/1.0 (for educational purposes only, contact: collen.ellis@sorbonne-universite.fr)",
        "Connection": "keep-alive"
    }

    # Parameters for the request
    params = {
        "q": f"pn={patent_number}",  # Ensure the format matches the EPO API
        "range": "1"  # Ensure this matches the range in your working URL
    }

    # Make the request
    response = requests.get(search_url, headers=headers, params=params)

    if response.status_code == 200:
        # Return the raw XML response text
        return response.text
    else:
        print(f"Failed to retrieve patent info for {patent_number}. Status code: {response.status_code}")
        return None

In [60]:
display(data_to_collect)

,year,country,doc_number
0,1782,GB,178201321
1,1784,GB,178401432
2,1785,GB,178501470
3,1786,GB,178601565
4,1787,GB,178701616
...,...,...,...
21448,2020,CA,3110896
21449,2020,IL,274993
21450,2020,CY,1122076
21451,2020,CN,211042545


In [66]:
def parse_xml(xml_data):
    
    # Parse the XML response
    root = ET.fromstring(xml_data)

    # Define the namespaces
    namespaces = {
        'ops': 'http://ops.epo.org',
        '': 'http://www.epo.org/exchange'  # Default namespace
    }

    # Extract all inventor names
    inventor_names = []
    for inventor in root.findall('.//parties//inventor-name', namespaces):
        name = inventor.find('name', namespaces).text
        if name:
            inventor_names.append(name.strip())

    # Extract the invention title
    title = root.find('.//invention-title', namespaces).text.strip() if root.find('.//invention-title', namespaces) is not None else ""

    # Extract all classification-ipcr entries
    classifications = []
    for classification in root.findall('.//classifications-ipcr//classification-ipcr', namespaces):
        classification_text = classification.find('text', namespaces).text.strip() if classification.find('text', namespaces) is not None else ""
        if classification_text:
            classifications.append(classification_text)

    return inventor_names, title, classifications


In [ ]:
def classify_ipc(ipc_code):
    # Simplified mapping of IPC codes to general categories
    ipc_category_map = {
        'A': 'Human necessities (e.g., Medicine, Food, Hygiene)',
        'B': 'Performing Operations; Transporting',
        'C': 'Chemistry; Metallurgy',
        'D': 'Textiles; Paper',
        'E': 'Fixed constructions',
        'F': 'Mechanical engineering; Lighting; Heating; Weapons',
        'G': 'Physics',
        'H': 'Electricity'
    }

    # Extract the first character of the IPC code to map to a category
    section = ipc_code[0]
    
    # Return the general category for that section
    return ipc_category_map.get(section, "Unknown Category")

def parse_classifications(classifications):
    meaningful_classifications = []
    for classification in classifications:
        ipc_code = classification.split()[0]  # Get the IPC code part
        category = classify_ipc(ipc_code)
        meaningful_classifications.append(f"{classification}: {category}")
    
    return meaningful_classifications




C21D   8/    02            A I: Chemistry; Metallurgy
C23C   2/    06            A I: Chemistry; Metallurgy
C23C   2/    12            A I: Chemistry; Metallurgy
C23C   2/    28            A I: Chemistry; Metallurgy
C25D   5/    36            A I: Chemistry; Metallurgy


In [ ]:

token = get_token()

# Iterate over a specific range of patents
for i in range(0, 10):
    if i % 100 == 0:
        token = get_token()

    row = data_to_collect.iloc[i]
    patent_number = row['doc_number']

    # Get patent info using the patent number and token
    patent_info = get_patent_info(patent_number, token)

    if patent_info is not None:
        # Parse the XML response
        xml_data = patent_info
        inventor_names, title, classifications = parse_xml(xml_data)

        meaningful_classifications = parse_classifications(classifications)

        print("Inventors:", inventor_names)
        print("Title:", title)
        print("Classifications:", meaningful_classifications)

    





Inventors: ['WATT JAMES\u2002[GB]']
Title: Steam engines
Classifications: []
Inventors: ['WATT JAMES\u2002[GB]']
Title: Fire and steam engines
Classifications: []
Inventors: []
Title: 
Classifications: []
Inventors: []
Title: 
Classifications: []
Inventors: []
Title: 
Classifications: []
Inventors: []
Title: 
Classifications: []
Inventors: ['BREWSTER DAVID\u2002[GB]']
Title: KALEIDOSCOPES
Classifications: []
Inventors: ['БОРИСЕНКО ІВАН І\u2002[UA]', 'КОХАСЬ ОЛЕКСАНД&\u2002[UA]', 'САВЧЕНКО ПЕТРО П\u2002[UA]', 'Борисенко Іван Ілліч,', 'Кохась Олександр Миколайович,', 'Савченко Петро Павлович']
Title: АГРЕГАТ СИЛОВИЙ УНІВЕРСАЛЬНИЙ
Classifications: []
Inventors: ['КОЧЕГАРОВ ЮРІЙ С\u2002[UA]', 'Кочегаров Юрій Сергійович']
Title: АПАРАТ ТЕЛЕФОННИЙ
Classifications: []
Inventors: ['ЛЕФЕ ПАТРІС СТЕФ\u2002[CH]', 'ЛЕФЕ Патріс Стефан Ернест']
Title: ІНФОРМАЦІЙНА ПАНЕЛЬ ТУРИСТИЧНИХ АВТОБУСІВ
Classifications: []
